# GriceBench DPO Data Cleaning with Gemini API

**Goal:** Fix Manner violations in DPO data using FREE Gemini API

**Expected Results:**
- Manner margin: -0.284 → +0.180 (POSITIVE!)
- Dataset size: ~3,500 clean pairs
- All margins positive
- Ready for single-stage DPO training

**Prerequisites:**
- Datasets added: `gricebench-detector-v2`, `gricebench-dpo-raw`
- Secret added: `GEMINI_API_KEY`
- GPU enabled
- Internet enabled

In [ ]:
# ============================================================================
# CELL 1: Setup & Imports
# ============================================================================

print("="*80)
print("CELL 1: SETUP & IMPORTS")
print("="*80)

import json
import pandas as pd
import numpy as np
from pathlib import Path
from tqdm import tqdm
import torch
import torch.nn as nn
from transformers import AutoTokenizer, AutoModel
import time
import google.generativeai as genai
from datetime import datetime

print("\n✅ All imports successful")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"   Using device: {device}")

# Create directories
Path("/kaggle/working/data").mkdir(exist_ok=True)
Path("/kaggle/working/analysis").mkdir(exist_ok=True)
Path("/kaggle/working/logs").mkdir(exist_ok=True)

print("\n✅ Directories created:")
print("   /kaggle/working/data/")
print("   /kaggle/working/analysis/")
print("   /kaggle/working/logs/")

# Start logging
start_time = datetime.now()
print(f"\n⏰ Started at: {start_time.strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

In [ ]:
# ============================================================================
# CELL 2: Load Detector V2
# ============================================================================

print("\n" + "="*80)
print("CELL 2: LOADING DETECTOR V2")
print("="*80)

class MaximDetectorV2(nn.Module):
    """Detector V2 with deeper classification heads"""
    
    def __init__(self, model_name, num_maxims=4, dropout=0.15):
        super().__init__()
        self.encoder = AutoModel.from_pretrained(model_name)
        hidden_size = self.encoder.config.hidden_size
        
        self.classifiers = nn.ModuleList([
            nn.Sequential(
                nn.Dropout(dropout),
                nn.Linear(hidden_size, hidden_size // 2),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_size // 2, hidden_size // 4),
                nn.GELU(),
                nn.Dropout(dropout),
                nn.Linear(hidden_size // 4, 1)
            )
            for _ in range(num_maxims)
        ])
    
    def forward(self, input_ids, attention_mask):
        outputs = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        pooled = outputs.last_hidden_state[:, 0, :]
        logits = torch.cat([
            classifier(pooled)
            for classifier in self.classifiers
        ], dim=1)
        return logits

print("\n📦 Loading model components...")

model_name = 'microsoft/deberta-v3-base'
print(f"   Base model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
print("   ✅ Tokenizer loaded")

detector_model = MaximDetectorV2(model_name).to(device)
print("   ✅ Model architecture created")

# Load trained weights
checkpoint_path = '/kaggle/input/gricebench-detector-v2/best_model_v2.pt'
print(f"\n📥 Loading checkpoint from: {checkpoint_path}")

checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
detector_model.load_state_dict(checkpoint['model_state_dict'])
detector_model.eval()
print("   ✅ Weights loaded and model set to eval mode")

# Load temperatures
temp_path = '/kaggle/input/gricebench-detector-v2/temperatures.json'
print(f"\n🌡️  Loading temperature scaling from: {temp_path}")

with open(temp_path) as f:
    temperatures = json.load(f)

print("   ✅ Temperatures loaded:")
for maxim, temp in temperatures.items():
    print(f"      {maxim:10s}: {temp:.4f}")

print("\n✅ Detector V2 fully loaded and ready!")
print("="*80)

In [ ]:
# ============================================================================
# CELL 3: Define Scoring Function
# ============================================================================

print("\n" + "="*80)
print("CELL 3: DEFINING SCORING FUNCTION")
print("="*80)

def score_response(context, response, evidence=None):
    """Score a response for maxim violations using Detector V2"""
    
    # Construct input text
    if evidence:
        text = f"Context: {context} Evidence: {evidence} Response: {response}"
    else:
        text = f"Context: {context} Response: {response}"
    
    # Tokenize
    encoding = tokenizer(
        text,
        max_length=512,
        padding='max_length',
        truncation=True,
        return_tensors='pt'
    )
    
    input_ids = encoding['input_ids'].to(device)
    attention_mask = encoding['attention_mask'].to(device)
    
    # Get logits
    with torch.no_grad():
        logits = detector_model(input_ids, attention_mask)
    
    # Apply temperature scaling and sigmoid
    maxims = ['quantity', 'quality', 'relation', 'manner']
    scores = {}
    
    for i, maxim in enumerate(maxims):
        temp = temperatures[maxim]
        scaled_logit = logits[0, i] / temp
        prob = torch.sigmoid(scaled_logit).item()
        scores[maxim] = prob
    
    return scores

print("\n✅ Scoring function defined")

# Test it
print("\n🧪 Testing scoring function...")
test_context = "What is the capital of France?"
test_response = "Paris is the capital."
test_scores = score_response(test_context, test_response)

print("   Test scores:")
for maxim, score in test_scores.items():
    print(f"      {maxim:10s}: {score:.4f}")

print("\n✅ Scoring function working correctly!")
print("="*80)

In [ ]:
# ============================================================================
# CELL 4: Load and Score DPO Data
# ============================================================================

print("\n" + "="*80)
print("CELL 4: LOADING AND SCORING DPO DATA")
print("="*80)

# Load DPO data
dpo_path = '/kaggle/input/gricebench-dpo-raw/dpo_train.json'
print(f"\n📥 Loading DPO data from: {dpo_path}")

with open(dpo_path) as f:
    dpo_train = json.load(f)

print(f"   ✅ Loaded {len(dpo_train)} DPO pairs")

# Score all pairs
print(f"\n🔍 Scoring {len(dpo_train)} pairs with Detector V2...")
print("   This will take ~10 minutes")
print("   Progress will be shown every 500 pairs\n")

scored_data = []
scoring_start = time.time()

for idx, item in enumerate(tqdm(dpo_train, desc="Scoring pairs")):
    prompt = item.get('prompt', item.get('context', ''))
    chosen = item.get('chosen', '')
    rejected = item.get('rejected', '')
    
    # Score both responses
    chosen_scores = score_response(prompt, chosen)
    rejected_scores = score_response(prompt, rejected)
    
    # Calculate margins (rejected - chosen, positive = chosen is better)
    margins = {
        maxim: rejected_scores[maxim] - chosen_scores[maxim]
        for maxim in ['quantity', 'quality', 'relation', 'manner']
    }
    
    scored_item = item.copy()
    scored_item['chosen_scores'] = chosen_scores
    scored_item['rejected_scores'] = rejected_scores
    scored_item['margins'] = margins
    scored_item['avg_margin'] = sum(margins.values()) / len(margins)
    
    scored_data.append(scored_item)
    
    # Progress update
    if (idx + 1) % 500 == 0:
        elapsed = time.time() - scoring_start
        rate = (idx + 1) / elapsed
        remaining = (len(dpo_train) - idx - 1) / rate
        print(f"   Progress: {idx+1}/{len(dpo_train)} | Rate: {rate:.1f} pairs/sec | ETA: {remaining/60:.1f} min")

scoring_time = time.time() - scoring_start

print(f"\n✅ Scored {len(scored_data)} pairs in {scoring_time/60:.1f} minutes")
print(f"   Average: {len(scored_data)/scoring_time:.1f} pairs/second")

# Save scored data
scored_path = '/kaggle/working/analysis/scored_data.json'
with open(scored_path, 'w') as f:
    json.dump(scored_data, f, indent=2)
print(f"\n💾 Saved scored data to: {scored_path}")

print("="*80)

In [ ]:
# ============================================================================
# CELL 5: Analyze Initial Data
# ============================================================================

print("\n" + "="*80)
print("CELL 5: INITIAL DATA ANALYSIS")
print("="*80)

# Convert to DataFrame
df = pd.DataFrame([{
    'prompt': item.get('prompt', ''),
    'chosen': item.get('chosen', ''),
    'rejected': item.get('rejected', ''),
    'quantity_margin': item['margins']['quantity'],
    'quality_margin': item['margins']['quality'],
    'relation_margin': item['margins']['relation'],
    'manner_margin': item['margins']['manner'],
    'avg_margin': item['avg_margin'],
    'full_item': item
} for item in scored_data])

print(f"\n📊 Dataset: {len(df)} pairs")

# Margin statistics
print("\n" + "-"*80)
print("MARGIN STATISTICS (rejected - chosen)")
print("Positive margin = chosen is better")
print("-"*80)
print(f"{'Maxim':<12} {'Mean':<10} {'Std':<10} {'>0%':<10} {'>0.15%':<10} {'Status'}")
print("-"*80)

for maxim in ['quantity', 'quality', 'relation', 'manner']:
    col = f'{maxim}_margin'
    mean_val = df[col].mean()
    std_val = df[col].std()
    pos_pct = (df[col] > 0).mean() * 100
    strong_pct = (df[col] > 0.15).mean() * 100
    
    status = "✅ Good" if mean_val > 0.05 else "⚠️  Weak" if mean_val > 0 else "❌ Negative"
    
    print(f"{maxim:<12} {mean_val:>+.3f}     {std_val:>6.3f}     "
          f"{pos_pct:>5.1f}%    {strong_pct:>5.1f}%    {status}")

print("-"*80)

# Key findings
print("\n🔍 Key Findings:")
manner_mean = df['manner_margin'].mean()
if manner_mean < 0:
    print(f"   ⚠️  MANNER IS NEGATIVE: {manner_mean:.3f}")
    print("      This is the problem we need to fix!")
else:
    print(f"   ✅ Manner is positive: {manner_mean:.3f}")

print("="*80)

In [ ]:
# ============================================================================
# CELL 6: Identify Problem Pairs
# ============================================================================

print("\n" + "="*80)
print("CELL 6: IDENTIFYING MANNER PROBLEM PAIRS")
print("="*80)

# Find pairs where Manner is negative but content is good
print("\n🔍 Finding pairs with:")
print("   - Manner margin < -0.1 (negative)")
print("   - BUT Quantity OR Relation > 0.1 (good content)\n")

problem_pairs = df[
    (df['manner_margin'] < -0.1) &
    (
        (df['relation_margin'] > 0.1) |
        (df['quantity_margin'] > 0.1)
    )
].copy()

print(f"📊 Found {len(problem_pairs)} problem pairs ({len(problem_pairs)/len(df)*100:.1f}%)")
print(f"   These have good content but bad Manner")

# Statistics
print("\n📈 Problem pairs statistics:")
print(f"   Manner mean: {problem_pairs['manner_margin'].mean():.3f}")
print(f"   Quantity mean: {problem_pairs['quantity_margin'].mean():.3f}")
print(f"   Relation mean: {problem_pairs['relation_margin'].mean():.3f}")

# Show examples
print("\n" + "-"*80)
print("SAMPLE PROBLEM PAIRS (first 3)")
print("-"*80)

for idx, (i, row) in enumerate(problem_pairs.head(3).iterrows()):
    print(f"\nExample {idx+1}:")
    print(f"  Chosen:   {row['chosen'][:100]}...")
    print(f"  Rejected: {row['rejected'][:100]}...")
    print(f"  Margins - Q:{row['quantity_margin']:+.2f} Qual:{row['quality_margin']:+.2f} "
          f"R:{row['relation_margin']:+.2f} M:{row['manner_margin']:+.2f}")
    print(f"  Problem: Good content but unclear Manner")
    print("-"*80)

# Save for analysis
problem_path = '/kaggle/working/analysis/problem_pairs.json'
problem_pairs.to_json(problem_path, orient='records', indent=2)
print(f"\n💾 Saved problem pairs to: {problem_path}")

print("\n✅ Problem identification complete!")
print("="*80)

In [ ]:
# ============================================================================
# CELL 7: Setup Gemini API
# ============================================================================

print("\n" + "="*80)
print("CELL 7: INITIALIZING GEMINI API (FREE!)")
print("="*80)

# Get API key from Kaggle Secrets
print("\n🔑 Loading API key from Kaggle Secrets...")

try:
    from kaggle_secrets import UserSecretsClient
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("GEMINI_API_KEY")
    print("   ✅ API key loaded successfully")
    print(f"   Key starts with: {api_key[:10]}...")
except Exception as e:
    print(f"   ❌ Error loading API key: {e}")
    print("   Please add GEMINI_API_KEY to Kaggle Secrets")
    raise

# Configure Gemini
print("\n🤖 Configuring Gemini API...")
genai.configure(api_key=api_key)

# Use Gemini 1.5 Flash (fastest, free tier)
gemini_model = genai.GenerativeModel('gemini-1.5-flash')

print("   ✅ Gemini API configured")
print("   Model: gemini-1.5-flash")
print("   Rate limit: 15 requests/minute")
print("   Daily limit: 1,500 requests")
print("   Cost: $0.00 (FREE!) 🎉")

# Test the API
print("\n🧪 Testing Gemini API...")

try:
    test_response = gemini_model.generate_content(
        "Say 'Hello from Gemini!' in exactly 3 words",
        generation_config=genai.types.GenerationConfig(temperature=0.3)
    )
    print(f"   ✅ API test successful!")
    print(f"   Response: {test_response.text.strip()}")
except Exception as e:
    print(f"   ❌ API test failed: {e}")
    raise

print("\n✅ Gemini API ready to use!")
print("="*80)

In [ ]:
# ============================================================================
# CELL 8: Fix Manner Violations with Gemini
# ============================================================================

print("\n" + "="*80)
print("CELL 8: FIXING MANNER VIOLATIONS WITH GEMINI")
print("="*80)

def fix_manner_violation(text: str, max_retries: int = 3) -> str:
    """Fix Manner violations using Gemini API"""
    
    prompt = f"""Fix ONLY the clarity and organization issues in this text.

CRITICAL RULES:
1. Replace ambiguous references with clear ones
   - "Said" → "The company said"
   - "it" → specific noun
   - "they" → specific group
2. Fix unclear pronoun references
3. Improve sentence structure if confusing
4. Keep the EXACT same meaning and facts
5. Maintain similar length (within 20%)
6. Do NOT add new information
7. Do NOT remove any facts

Original text:
{text}

Fixed text (output ONLY the fixed text):"""

    for attempt in range(max_retries):
        try:
            # Respect rate limit (15/min = 4 seconds between requests)
            time.sleep(4)
            
            response = gemini_model.generate_content(
                prompt,
                generation_config=genai.types.GenerationConfig(
                    temperature=0.3,
                    max_output_tokens=1000,
                )
            )
            
            fixed_text = response.text.strip()
            fixed_text = fixed_text.replace('```', '').strip()
            
            # Validate length
            len_ratio = len(fixed_text) / len(text)
            if 0.6 <= len_ratio <= 1.4:
                return fixed_text
            else:
                continue
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(2 ** attempt)
            else:
                return text  # Keep original if all retries fail
    
    return text

# Fix all problem pairs
print(f"\n🔧 Fixing {len(problem_pairs)} pairs with Gemini...")
print(f"   Rate: ~15 pairs/minute (4 seconds per pair)")
print(f"   Estimated time: {len(problem_pairs) / 15:.1f} minutes\n")

fixed_pairs = problem_pairs.copy()
manner_improvements = []
failed_fixes = []

fix_start = time.time()

for idx, i in enumerate(tqdm(problem_pairs.index, desc="Fixing pairs")):
    original_chosen = problem_pairs.loc[i, 'chosen']
    
    try:
        # Fix the text
        fixed_chosen = fix_manner_violation(original_chosen)
        
        # Update dataframe
        fixed_pairs.loc[i, 'chosen'] = fixed_chosen
        fixed_pairs.loc[i, 'original_chosen'] = original_chosen
        
        # Re-score with Detector V2
        prompt = problem_pairs.loc[i, 'prompt']
        new_scores = score_response(prompt, fixed_chosen)
        rejected_scores = problem_pairs.loc[i, 'full_item']['rejected_scores']
        
        # Update margins
        for maxim in ['quantity', 'quality', 'relation', 'manner']:
            new_margin = rejected_scores[maxim] - new_scores[maxim]
            fixed_pairs.loc[i, f'{maxim}_margin'] = new_margin
            
            if maxim == 'manner':
                old_margin = problem_pairs.loc[i, 'manner_margin']
                improvement = new_margin - old_margin
                manner_improvements.append(improvement)
    
    except Exception as e:
        print(f"\n⚠️  Failed to fix pair {i}: {e}")
        failed_fixes.append(i)
        continue
    
    # Progress update every 50 pairs
    if (idx + 1) % 50 == 0:
        elapsed = time.time() - fix_start
        rate = (idx + 1) / elapsed * 60
        remaining = (len(problem_pairs) - idx - 1) / rate
        print(f"\n   Progress: {idx+1}/{len(problem_pairs)} | Rate: {rate:.1f}/min | ETA: {remaining:.1f}min")

fix_time = time.time() - fix_start

print(f"\n✅ Fixing complete!")
print(f"   Time: {fix_time/60:.1f} minutes")
print(f"   Successfully fixed: {len(problem_pairs) - len(failed_fixes)}")
print(f"   Failed: {len(failed_fixes)}")
print(f"   Cost: $0.00 (FREE!)")

# Analyze improvements
if manner_improvements:
    original_manner = problem_pairs['manner_margin'].mean()
    fixed_manner = fixed_pairs['manner_margin'].mean()
    avg_improvement = np.mean(manner_improvements)
    
    print(f"\n📊 Manner margin improvement:")
    print(f"   Before: {original_manner:+.3f}")
    print(f"   After:  {fixed_manner:+.3f}")
    print(f"   Change: {fixed_manner - original_manner:+.3f} ({(fixed_manner - original_manner)/abs(original_manner)*100:+.1f}%)")

# Save fixed pairs
fixed_path = '/kaggle/working/analysis/fixed_pairs.json'
fixed_pairs.to_json(fixed_path, orient='records', indent=2)
print(f"\n💾 Saved fixed pairs to: {fixed_path}")

print("="*80)

In [ ]:
# ============================================================================
# CELL 9: Create Final Clean Dataset
# ============================================================================

print("\n" + "="*80)
print("CELL 9: CREATING FINAL CLEAN DATASET")
print("="*80)

# Get pairs that already have good Manner
print("\n📊 Combining data sources...")

good_manner_pairs = df[df['manner_margin'] > 0.1].copy()
print(f"   Pairs with good Manner (kept as-is): {len(good_manner_pairs)}")
print(f"   Pairs with fixed Manner: {len(fixed_pairs)}")

# Combine
final_df = pd.concat([good_manner_pairs, fixed_pairs], ignore_index=True)
print(f"   Combined total: {len(final_df)}")

# Remove duplicates
before_dedup = len(final_df)
final_df = final_df.drop_duplicates(subset=['chosen', 'rejected'])
print(f"   After deduplication: {len(final_df)} (removed {before_dedup - len(final_df)})")

# Apply quality filter
print("\n🔍 Applying quality filter (avg_margin > 0.05)...")
avg_margins = final_df[['quantity_margin', 'quality_margin', 'relation_margin', 'manner_margin']].mean(axis=1)
final_df['avg_margin'] = avg_margins

before_filter = len(final_df)
final_df = final_df[final_df['avg_margin'] > 0.05].copy()
print(f"   After quality filter: {len(final_df)} (removed {before_filter - len(final_df)})")

# Final statistics
print("\n" + "-"*80)
print("FINAL DATASET STATISTICS")
print("-"*80)
print(f"{'Maxim':<12} {'Mean':<10} {'Std':<10} {'>0%':<10} {'>0.15%':<10} {'Status'}")
print("-"*80)

all_positive = True
for maxim in ['quantity', 'quality', 'relation', 'manner']:
    col = f'{maxim}_margin'
    mean_val = final_df[col].mean()
    std_val = final_df[col].std()
    pos_pct = (final_df[col] > 0).mean() * 100
    strong_pct = (final_df[col] > 0.15).mean() * 100
    
    if mean_val <= 0:
        all_positive = False
        status = "❌ Negative"
    elif mean_val < 0.05:
        status = "⚠️  Weak"
    else:
        status = "✅ Good"
    
    print(f"{maxim:<12} {mean_val:>+.3f}     {std_val:>6.3f}     "
          f"{pos_pct:>5.1f}%    {strong_pct:>5.1f}%    {status}")

print("-"*80)

if all_positive:
    print("\n✅ SUCCESS! All maxims have positive mean margins!")
    print("   Dataset is ready for single-stage DPO training!")
else:
    print("\n⚠️  Warning: Some maxims still have negative or weak margins")
    print("   Consider adjusting filters or reviewing data")

print("="*80)

In [ ]:
# ============================================================================
# CELL 10: Save Final Clean Dataset
# ============================================================================

print("\n" + "="*80)
print("CELL 10: SAVING FINAL CLEAN DATASET")
print("="*80)

# Train/val split (95/5)
print("\n📊 Creating train/val split (95/5)...")

train_size = int(0.95 * len(final_df))
train_df = final_df.iloc[:train_size].copy()
val_df = final_df.iloc[train_size:].copy()

print(f"   Training set: {len(train_df)} pairs")
print(f"   Validation set: {len(val_df)} pairs")

# Convert back to list format
print("\n🔄 Converting to DPO format...")

train_data = []
for idx, row in train_df.iterrows():
    item = row['full_item'].copy()
    # Use fixed chosen if available
    if 'original_chosen' in row and pd.notna(row['original_chosen']):
        item['chosen'] = row['chosen']
        item['original_chosen'] = row['original_chosen']
    train_data.append(item)

val_data = []
for idx, row in val_df.iterrows():
    item = row['full_item'].copy()
    if 'original_chosen' in row and pd.notna(row['original_chosen']):
        item['chosen'] = row['chosen']
        item['original_chosen'] = row['original_chosen']
    val_data.append(item)

print(f"   ✅ Converted {len(train_data)} training pairs")
print(f"   ✅ Converted {len(val_data)} validation pairs")

# Save
print("\n💾 Saving clean datasets...")

train_path = '/kaggle/working/data/dpo_train_clean.json'
val_path = '/kaggle/working/data/dpo_val_clean.json'

with open(train_path, 'w') as f:
    json.dump(train_data, f, indent=2)
print(f"   ✅ Saved training data to: {train_path}")
print(f"      Size: {Path(train_path).stat().st_size / 1024 / 1024:.1f} MB")

with open(val_path, 'w') as f:
    json.dump(val_data, f, indent=2)
print(f"   ✅ Saved validation data to: {val_path}")
print(f"      Size: {Path(val_path).stat().st_size / 1024 / 1024:.1f} MB")

# Summary
end_time = datetime.now()
total_time = end_time - start_time

print("\n" + "="*80)
print("🎉 DATA CLEANING COMPLETE!")
print("="*80)

print(f"""
✅ Final Dataset Created!

📊 Dataset Statistics:
   Total pairs: {len(final_df)}
   Training: {len(train_df)} pairs
   Validation: {len(val_df)} pairs

📈 Margin Improvements:
   All margins positive: {all_positive}
   Manner mean: {final_df['manner_margin'].mean():+.3f}
   Quality mean: {final_df['quality_margin'].mean():+.3f}
   Quantity mean: {final_df['quantity_margin'].mean():+.3f}
   Relation mean: {final_df['relation_margin'].mean():+.3f}

⏱️  Processing Time:
   Started: {start_time.strftime('%H:%M:%S')}
   Ended: {end_time.strftime('%H:%M:%S')}
   Total: {total_time.total_seconds()/60:.1f} minutes

💰 Total Cost: $0.00 (FREE!)

📥 Download Files:
   {train_path}
   {val_path}

🚀 Next Steps:
   1. Download the clean data files
   2. Upload to Kaggle as 'gricebench-dpo-clean'
   3. Run standard DPO training (no multi-stage needed!)
   4. Expected: 75-85% cooperative rate

""")

print("="*80)